In [1]:
import pandas as pd
import numpy as np

In [2]:
ratings = pd.read_csv('final_rating.csv', sep=",", on_bad_lines='skip', encoding='Latin-1')
ratings

,Unnamed: 0,title,num_of_rating,user_id,ISBN,rating,author,year,Publisher,Image_URL
0,290,10 Lb. Penalty,35,39281,0515123471,0,Dick Francis,1998,Jove Books,http://images.amazon.com/images/P/0515123471.0...
1,291,10 Lb. Penalty,35,49109,0515123471,5,Dick Francis,1998,Jove Books,http://images.amazon.com/images/P/0515123471.0...
2,292,10 Lb. Penalty,35,69405,0515123471,0,Dick Francis,1998,Jove Books,http://images.amazon.com/images/P/0515123471.0...
3,293,10 Lb. Penalty,35,76352,0515123471,0,Dick Francis,1998,Jove Books,http://images.amazon.com/images/P/0515123471.0...
4,294,10 Lb. Penalty,35,83287,0515123471,10,Dick Francis,1998,Jove Books,http://images.amazon.com/images/P/0515123471.0...
...,...,...,...,...,...,...,...,...,...,...
93853,487432,"\O\"" Is for Outlaw""",105,234359,0805059555,0,Sue Grafton,1999,Henry Holt &amp; Company,http://images.amazon.com/images/P/0805059555.0...
93854,487433,"\O\"" Is for Outlaw""",105,234623,0805059555,0,Sue Grafton,1999,Henry Holt &amp; Company,http://images.amazon.com/images/P/0805059555.0...
93855,487434,"\O\"" Is for Outlaw""",105,236283,0805059555,10,Sue Grafton,1999,Henry Holt &amp; Company,http://images.amazon.com/images/P/0805059555.0...
93856,487435,"\O\"" Is for Outlaw""",105,238120,0805059555,0,Sue Grafton,1999,Henry Holt &amp; Company,http://images.amazon.com/images/P/0805059555.0...


In [3]:
users = pd.read_csv('bookData/Users.csv', sep=",", on_bad_lines='skip', encoding='Latin-1')

In [4]:
users.rename(columns = {
    'User-ID': 'user_id',
}, inplace = True)

In [5]:
user_merge_ratings = users.merge(ratings, on = "user_id")
user_merge_ratings

,user_id,Location,Age,Unnamed: 0,title,num_of_rating,ISBN,rating,author,year,Publisher,Image_URL
0,254,"minneapolis, minnesota, usa",24.0,1073,1984,76,0451524934,9,George Orwell,1990,Signet Book,http://images.amazon.com/images/P/0451524934.0...
1,254,"minneapolis, minnesota, usa",24.0,3164,A Bend in the Road,115,0446527785,0,Nicholas Sparks,2001,Warner Books,http://images.amazon.com/images/P/0446527785.0...
2,254,"minneapolis, minnesota, usa",24.0,18597,ANGELA'S ASHES,81,0684874350,0,Frank McCourt,1996,Scribner,http://images.amazon.com/images/P/0684874350.0...
3,254,"minneapolis, minnesota, usa",24.0,27152,American Gods,94,0380789035,10,Neil Gaiman,2002,HarperTorch,http://images.amazon.com/images/P/0380789035.0...
4,254,"minneapolis, minnesota, usa",24.0,30289,Angela's Ashes (MMP) : A Memoir,92,0684872153,0,Frank McCourt,1999,Scribner,http://images.amazon.com/images/P/0684872153.0...
...,...,...,...,...,...,...,...,...,...,...,...,...
93853,278418,"omaha, nebraska, usa",NaN,472933,Where the Heart Is (Oprah's Book Club (Paperba...,183,0446672211,0,Billie Letts,1998,Warner Books,http://images.amazon.com/images/P/0446672211.0...
93854,278418,"omaha, nebraska, usa",NaN,473151,Where the Red Fern Grows,62,0553274295,0,Wilson Rawls,1984,Random House Children's Books,http://images.amazon.com/images/P/0553274295.0...
93855,278418,"omaha, nebraska, usa",NaN,473789,"Whirlwind (Tyler, Book 1)",78,0373825013,0,Nancy Martin,1992,Harlequin,http://images.amazon.com/images/P/0373825013.0...
93856,278418,"omaha, nebraska, usa",NaN,478523,Windmills of the Gods,43,0446350109,0,Sidney Sheldon,1987,Warner Books,http://images.amazon.com/images/P/0446350109.0...


In [6]:
user_merge_ratings = user_merge_ratings[user_merge_ratings['num_of_rating'] >= 50]

In [7]:
regions_index = user_merge_ratings['Location'].unique()

In [8]:
region_unique = []
for i in range(regions_index.size):
    temp = regions_index[i].split(",")
    single_temp = temp[len(temp) - 1].strip()
    if single_temp in region_unique or single_temp == '':
        continue
    else :
        region_unique.append(single_temp)

region_unique

['usa',
 'spain',
 'canada',
 'n/a',
 'australia',
 'malaysia',
 'united kingdom',
 'portugal',
 'netherlands',
 'new zealand',
 'france',
 'switzerland',
 'germany',
 'austria',
 'italy',
 'china',
 'qatar',
 'finland',
 'japan',
 'sweden',
 'iran',
 'denmark',
 'brazil',
 'romania',
 'us',
 'belgium',
 'dominican republic']

In [9]:
import pickle
with open('region_unique.pkl', 'wb') as file:
    pickle.dump(region_unique, file)    